## Requirements & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -q -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Config & Training 

In [3]:
from huggingface_hub import hf_hub_download

realistic_vision_path = hf_hub_download(repo_id="SG161222/Realistic_Vision_V5.1_noVAE", filename="Realistic_Vision_V5.1-inpainting.safetensors")
vae_path = hf_hub_download(repo_id="stabilityai/sd-vae-ft-mse-original", filename="vae-ft-mse-840000-ema-pruned.safetensors")

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HNXIsW3WwnidNDQZHexX")
project = rf.workspace("arked").project("kvist_windows")
dataset = project.version(3).download("yolov7")



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to kvist_windows-3 in yolov7pytorch:: 100%|██████████| 83/83 [00:00<00:00, 741.28it/s]


In [5]:
import ml_collections

config = ml_collections.ConfigDict()

## General
config.seed = 2808
config.logdir = "logs"
config.device = 'cuda'

## Dataset
config.dataset = dataset = ml_collections.ConfigDict()
# Roboflow API key
dataset.roboflow_api_key = 'HNXIsW3WwnidNDQZHexX'
# Roboflow workspace
dataset.roboflow_workspace = 'arked'
# Roboflow project name
dataset.project_name = 'kvist_windows'
# Roboflow dataset version
dataset.dataset_version = 3
# Local path where the dataset is downloaded
dataset.data_root = 'data'
# image size
dataset.image_size = 512
# max image size
dataset.max_image_size = 512
# normalize images
dataset.normalize_images = True
# scaling pixels of the mask
dataset.scaling_pixels = 4

# Prompt
config.prompt = prompt = ml_collections.ConfigDict()
# Global caption to add to every prompt
prompt.global_caption = None
prompt.negative_caption = 'ugly, blurry, poor quality'

# Model
config.model = model = ml_collections.ConfigDict()
model.model_path = realistic_vision_path
model.vae_path = vae_path

## Training
config.train = train = ml_collections.ConfigDict()

# LoRA hyperparameters
train.lora_rank = 8
train.lora_scale = 1.0
train.lora_dropout_p = 0.1
train.train_unet = True
train.train_text_encoder = True
train.unet_lr = 2e-4
train.text_encoder_lr = 2e-4
train.mask_temperature = 1.0

# train batch size.
train.train_batch_size = 2
# eval batch size.
train.eval_batch_size = 1

# number of steps of the noise scheduler
train.num_train_timesteps = 50
# TODO add noise scheduler type and hyperparameters
""" 
{
  "_class_name": "DEISMultistepScheduler",
  "_diffusers_version": "0.16.1",
  "algorithm_type": "deis",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "lower_order_final": true,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "solver_order": 2,
  "solver_type": "logrho",
  "steps_offset": 1,
  "thresholding": false,
  "trained_betas": null
}

"""

# number of steps between evaluating and saving model checkpoints.
train.eval_every_n_epochs = 5
# number of checkpoints to keep before overwriting old ones.
train.num_checkpoint_limit = 5
# mixed precision training. options are "fp16", "bf16", and "no". half-precision speeds up training significantly.
train.mixed_precision = "no"
# whether to use gradient checkpointing to reduce memory usage.
train.gradient_checkpointing = False
# number of gradient accumulation steps.
train.gradient_accumulation_steps = 1
# whether to use a learning rate schedule.
train.use_scheduler = True
# learning rate schedule type.
train.scheduler_type = "cosine_with_restarts"
# num_cycles for the learning rate schedule.
train.scheduler_num_cycles = 2
# number of warmup steps for the learning rate schedule.
train.scheduler_warmup_steps = 500
# number of steps to train for.
train.total_steps = 4000
# learning rate.
train.learning_rate = 1e-4
# Adam beta1.
train.adam_beta1 = 0.9
# Adam beta2.
train.adam_beta2 = 0.999
# Adam weight decay.
train.adam_weight_decay = 1e-4
# Adam epsilon.
train.adam_epsilon = 1e-8
# clip gradients.
train.clip_gradients = True
# clip gradients max norm.
train.clip_gradients_max_norm = 1.0
# use xformers efficient attention
train.use_xformers = True
# checkpoint folder
train.checkpoint_folder = "checkpoints"

## Evaluation
config.eval = eval = ml_collections.ConfigDict()
# number of validation sample steps.
eval.num_eval_steps = 20
# whether to use a validation set.
eval.use_validation = True
# the prompts to use for validation.
eval.prompts = ['']
# the strengths to use for validation.
eval.strengths = [1.0]
# eval epochs number.
eval.eval_epochs = 10

## Wandb
# whether to use wandb.
config.log_wandb = True
# wandb project name.
config.wandb = wandb = ml_collections.ConfigDict()
# wandb project name.
wandb.project_name = "kvist_windows"
# wandb entity name.
wandb.entity_name = "arked"
# wandb run name. (if not set is assigned automatically)
wandb.run_name = None
# wandb tags.
wandb.tags = ["lora"]

In [6]:
from src.training import train

train(config)

The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling t

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to data in yolov7pytorch:: 100%|██████████| 83/83 [00:00<00:00, 1035.34it/s]


loading VAE...
loading model...


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maidacundo (arked). Use `wandb login --relogin` to force relogin


Unet LoRA params: 2492928
CLIP LoRA params: 589824
Directory 'checkpoints' already exists.


Steps:   0%|          | 2/4000 [01:02<36:29:52, 32.86s/it, loss_lora=0.000256, lr=8e-7]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 from src.training import train                                                               │
│   2                                                                                              │
│ ❱ 3 train(config)                                                                                │
│   4                                                                                              │
│                                                                                                  │
│ c:\Users\Facundo\Desktop\ARKED\training\inpainting_training\lora_inpainting\src\training.py:171  │
│ in train                                                                                         │
│                                                                                                  │
│   168 │   for epoch in range(math.ceil(config.train.total_steps / len(train_dataloader))):       │
│   169 │   │   for batch in train_dataloader:                                                     │
│   170 │   │   │   optimizer_lora.zero_grad()                                                     │
│ ❱ 171 │   │   │   loss_lora = loss_step(                                                         │
│   172 │   │   │   │   batch,                                                                     │
│   173 │   │   │   │   unet,                                                                      │
│   174 │   │   │   │   vae,                                                                       │
│                                                                                                  │
│ c:\Users\Facundo\Desktop\ARKED\training\inpainting_training\lora_inpainting\src\training.py:284  │
│ in loss_step                                                                                     │
│                                                                                                  │
│   281 │                                                                                          │
│   282 │   # scale the mask                                                                       │
│   283 │   mask = F.interpolate(                                                                  │
│ ❱ 284 │   │   │   │   batch["mask"].to(dtype=weight_dtype).to(unet.device),                      │
│   285 │   │   │   │   scale_factor=1 / 8,                                                        │
│   286 │   │   │   )                                                                              │
│   287                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt